In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import datetime,date
from scipy.signal import resample
import numpy as np
import glob
import re


Save like Walk/1_eye.csv, Walk/2_eye.csv

In [2]:
files=glob.glob(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\Sea\*\*\eye*")
len(files)

27

In [75]:
import pandas as pd
import re
from datetime import datetime

def preprocess_eye_tracking_data(file_path):
    eye = pd.read_csv(file_path)
    
    # Extract date from filename
    file_name = file_path.split('/')[-1]
    date_pattern = r'(\d{2}-[A-Za-z]{3}-\d{4})'
    match = re.search(date_pattern, file_name)

    if match:
        date_str = match.group(1)  
        # Convert date string to a datetime object
        date = datetime.strptime(date_str, '%d-%b-%Y')
        # Add the date column to the DataFrame
        eye['date'] = date
    else:
        print("Date not found in the filename.")
        return eye

    # Function to convert time string to datetime in 24-hour format
    def convert_time(time_str):
        try:
            # Replace '-' with ':' and format milliseconds
            time_str = time_str.replace('-', ':')
            # Ensure milliseconds are 3 digits
            parts = time_str.split(':')
            if len(parts) == 4:
                milliseconds = parts[3][:3].ljust(3, '0')
                time_str = f"{parts[0]}:{parts[1]}:{parts[2]}.{milliseconds}"
            # Convert to 24-hour format
            return pd.to_datetime(time_str, format='%H:%M:%S.%f', errors='coerce').strftime('%H:%M:%S.%f')
        except ValueError:
            return pd.NaT

    # Apply conversion
    eye['Time'] = eye['Time'].apply(convert_time)

    # Combine date and time into a single datetime column
    eye['datetime'] = pd.to_datetime(eye['date'].astype(str) + ' ' + eye['Time'])

    # # Drop the original Time column if needed
    # eye.drop(['Time'], axis=1, inplace=True)

    return eye


eye_data = preprocess_eye_tracking_data(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\Walk\head_and_eye_tracking_walk\03_132566064405832227\eye_tracking_data-31-Jan-2021-(04-47-49).csv")
eye_data


,Time,#Frame,ConvergenceValid,Convergence_distance,Left_Eye_Openness,Right_Eye_Openness,Left_Eye_Closed,Right_Eye_Closed,LeftPupilDiameter,RightPupilDiameter,...,NrmRightEyeOriginZ,NrmSRLeftEyeGazeDirX,NrmSRLeftEyeGazeDirY,NrmSRLeftEyeGazeDirZ,NrmSRRightEyeGazeDirX,NrmSRRightEyeGazeDirY,NrmSRRightEyeGazeDirZ,Unnamed: 40,date,datetime
0,04:41:14.574000,1249,True,1.001011,1.0,1.0,False,False,2.808716,3.080109,...,-25.51028,-0.011612,-0.064270,0.997864,0.049164,-0.056885,0.997162,NaN,2021-01-31,2021-01-31 04:41:14.574
1,04:41:14.710000,1250,True,1.111169,1.0,1.0,False,False,2.788910,3.147308,...,-24.09032,0.053757,-0.167633,0.984375,0.093597,-0.147476,0.984619,NaN,2021-01-31,2021-01-31 04:41:14.710
2,04:41:14.746000,1251,True,1.210112,1.0,1.0,False,False,2.788635,3.147385,...,-24.08829,0.051712,-0.167175,0.984558,0.093048,-0.149399,0.984375,NaN,2021-01-31,2021-01-31 04:41:14.746
3,04:41:14.797000,1252,True,1.041183,1.0,1.0,False,False,2.786819,3.153839,...,-24.07057,0.048065,-0.168350,0.984543,0.096771,-0.146561,0.984451,NaN,2021-01-31,2021-01-31 04:41:14.797
4,04:41:14.895000,1253,True,1.000872,1.0,1.0,False,False,2.843109,3.150696,...,-24.06320,0.048019,-0.167328,0.984726,0.097046,-0.145233,0.984619,NaN,2021-01-31,2021-01-31 04:41:14.895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10509,04:47:48.699000,11758,True,0.317285,1.0,1.0,False,False,3.177841,3.642807,...,-20.42726,0.217331,-0.191498,0.957123,0.219605,-0.163666,0.961746,NaN,2021-01-31,2021-01-31 04:47:48.699
10510,04:47:48.732000,11759,True,0.311199,1.0,1.0,False,False,3.168945,3.636688,...,-20.44287,0.217316,-0.192291,0.956955,0.219513,-0.164261,0.961670,NaN,2021-01-31,2021-01-31 04:47:48.732
10511,04:47:48.765000,11760,True,0.313469,1.0,1.0,False,False,3.160522,3.635742,...,-20.46075,0.216553,-0.193085,0.956970,0.219208,-0.163849,0.961807,NaN,2021-01-31,2021-01-31 04:47:48.765
10512,04:47:48.798000,11761,True,2.034974,1.0,1.0,False,False,3.153397,3.623001,...,-19.98398,0.216141,-0.194336,0.956818,0.234802,-0.181473,0.954941,NaN,2021-01-31,2021-01-31 04:47:48.798


In [86]:
import pandas as pd

def preprocess_fms_data(file_path):
    # Load the CSV file into a DataFrame
    all_fms = pd.read_csv(file_path)

    # Convert 'Time ' column to datetime
    all_fms['DateTime'] = pd.to_datetime(all_fms['Time '], format='%Y.%m.%d %H:%M:%S:%f', errors='coerce')

    # Extract Date and Time into separate columns
    all_fms['Date'] = all_fms['DateTime'].dt.date
    all_fms['Time'] = all_fms['DateTime'].dt.strftime('%I:%M:%S.%f')  # 24-hour format

    # Drop the original 'Time ' column
    all_fms.drop(['Time '], axis=1, inplace=True)

    # Combine Date and Time into a single datetime column
    all_fms['DateTime'] = pd.to_datetime(all_fms['Date'].astype(str) + ' ' + all_fms['Time'])


    return all_fms

# Example usage:
file_path = r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\Room\Anonymous_FMS_All.csv"
processed_fms = preprocess_fms_data(file_path)
processed_fms


,Participants Code,Frame,FMS,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,DateTime,Date,Time
0,3.0,6273.0,0.0,NaN,NaN,NaN,NaN,2021-01-31 03:56:35.024,2021-01-31,03:56:35.024000
1,NaN,7286.0,0.0,NaN,NaN,NaN,NaN,2021-01-31 03:57:07.038,2021-01-31,03:57:07.038000
2,NaN,8242.0,0.0,NaN,NaN,NaN,NaN,2021-01-31 03:57:39.046,2021-01-31,03:57:39.046000
3,NaN,9217.0,0.0,NaN,NaN,NaN,NaN,2021-01-31 03:58:11.047,2021-01-31,03:58:11.047000
4,NaN,10164.0,0.0,NaN,NaN,NaN,NaN,2021-01-31 03:58:43.083,2021-01-31,03:58:43.083000
...,...,...,...,...,...,...,...,...,...,...
372,NaN,9933.0,0.0,NaN,NaN,NaN,NaN,2021-04-06 08:16:16.997,2021-04-06,08:16:16.997000
373,NaN,10880.0,0.0,NaN,NaN,NaN,NaN,2021-04-06 08:16:48.996,2021-04-06,08:16:48.996000
374,NaN,11818.0,0.0,NaN,NaN,NaN,NaN,2021-04-06 08:17:20.997,2021-04-06,08:17:20.997000
375,NaN,12781.0,0.0,NaN,NaN,NaN,NaN,2021-04-06 08:17:53.008,2021-04-06,08:17:53.008000


In [84]:


# def preprocess_fms_data(file_path):
#     # Load the CSV file into a DataFrame
#     all_fms = pd.read_csv(file_path)

#     # Convert 'Time ' column to datetime
#     all_fms['DateTime'] = pd.to_datetime(all_fms['Time '], format='%Y.%m.%d %H:%M:%S:%f', errors='coerce')

#     # Extract Date and Time into separate columns
#     all_fms['Date'] = all_fms['DateTime'].dt.date
#     all_fms['Time'] = all_fms['DateTime'].dt.strftime('%I:%M:%S:%f')
    

#     # Drop the original 'Time ' column
#     all_fms.drop(['Time '], axis=1, inplace=True)
#     all_fms['DateTime'] = all_fms.apply(lambda row: pd.datetime.combine(row['Date'], row['Time']), axis=1)

    

#     return all_fms


# file_path = r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\Room\Anonymous_FMS_All.csv"
# processed_fms = preprocess_fms_data(file_path)
# processed_fms


AttributeError: module 'pandas' has no attribute 'datetime'

In [90]:
import pandas as pd

def match_eye_tracking_with_fms(eye, all_fms):
    # Ensure 'datetime' in eye data is a datetime object
    eye['datetime'] = pd.to_datetime(eye['datetime'])

    # Extract the date from the eye data
    eye['date'] = eye['datetime'].dt.date
    specific_date = eye['date'].iloc[0]

    # Filter the FMS data for the specific date
    filtered_fms = all_fms.loc[all_fms['Date'] == specific_date]

    # Determine the start and end times from the eye data
    start_time = eye['Time'].iloc[0]
    end_time = eye['Time'].iloc[-1]

    # Filter FMS data for matching times
    fms_p1 = filtered_fms[(filtered_fms['Time'] > start_time) & (filtered_fms['Time'] <= end_time)]

    # Optionally, get the unique participant code
    matching_participants = fms_p1['Participants Code'].dropna().unique()
    if len(matching_participants) > 0:
        eye['participant'] = matching_participants[0]

    # Match FMS data with eye tracking data
    for i in range(fms_p1.shape[0] - 1):  # Use -1 to avoid out-of-bounds error
        start = fms_p1.DateTime.iloc[i]
        end = fms_p1.DateTime.iloc[i + 1]
        
        for j in range(eye.shape[0]):
            if (i == 0 and eye['datetime'].iloc[j] >= (start - pd.Timedelta(seconds=31))):
                eye.at[j, 'FMS'] = fms_p1.FMS.iloc[i]
            elif (i > 0 and eye['datetime'].iloc[j] >= (start - pd.Timedelta(seconds=31)) and eye['datetime'].iloc[j] <= (start + pd.Timedelta(seconds=31))):
                eye.at[j, 'FMS'] = fms_p1.FMS.iloc[i]

    return eye, fms_p1

# Assuming these functions are defined somewhere else in your code
eye = preprocess_eye_tracking_data(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\Roller\head_and_eye_tracking_roller\02_132568647919622523\eye_tracking_data-03-Feb-2021-(04-34-05).csv")
all_fms = preprocess_fms_data(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\Roller\Anonymous_FMS_All.csv")

updated_eye,fms_p1 = match_eye_tracking_with_fms(eye, all_fms)


In [91]:
np.unique(updated_eye.FMS),np.unique(fms_p1.FMS)

(array([0., 1., 2., 3.]), array([0., 1., 2., 3.]))

In [92]:
updated_eye

,Time,#Frame,ConvergenceValid,Convergence_distance,Left_Eye_Openness,Right_Eye_Openness,Left_Eye_Closed,Right_Eye_Closed,LeftPupilDiameter,RightPupilDiameter,...,NrmSRLeftEyeGazeDirY,NrmSRLeftEyeGazeDirZ,NrmSRRightEyeGazeDirX,NrmSRRightEyeGazeDirY,NrmSRRightEyeGazeDirZ,Unnamed: 40,date,datetime,participant,FMS
0,04:27:04.933000,1137,False,0.000000,0.227992,0.274306,False,False,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2021-02-03,2021-02-03 04:27:04.933,2.0,0.0
1,04:27:04.997000,1138,False,0.000000,0.170843,0.117896,False,False,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2021-02-03,2021-02-03 04:27:04.997,2.0,0.0
2,04:27:05.044000,1139,False,0.000000,0.170244,0.121416,False,True,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2021-02-03,2021-02-03 04:27:05.044,2.0,0.0
3,04:27:05.078000,1140,False,0.000000,0.518460,0.571331,False,False,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2021-02-03,2021-02-03 04:27:05.078,2.0,0.0
4,04:27:05.110000,1141,False,0.000000,0.765172,0.806785,False,False,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2021-02-03,2021-02-03 04:27:05.110,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13602,04:34:04.984000,14739,True,1.132965,1.000000,1.000000,False,False,3.680328,3.185547,...,-0.151886,0.980438,-0.139633,-0.175659,0.974487,NaN,2021-02-03,2021-02-03 04:34:04.984,2.0,0.0
13603,04:34:05.013000,14740,True,1.141850,1.000000,1.000000,False,False,3.682358,3.195770,...,-0.146286,0.981399,-0.141876,-0.170639,0.975052,NaN,2021-02-03,2021-02-03 04:34:05.013,2.0,0.0
13604,04:34:05.045000,14741,True,1.414185,1.000000,1.000000,False,False,3.717407,3.191711,...,-0.103302,0.986465,-0.165283,-0.150055,0.974747,NaN,2021-02-03,2021-02-03 04:34:05.045,2.0,0.0
13605,04:34:05.077000,14742,True,0.350547,1.000000,1.000000,False,False,3.725693,3.205002,...,-0.089783,0.985580,-0.165298,-0.143967,0.975677,NaN,2021-02-03,2021-02-03 04:34:05.077,2.0,0.0


In [96]:

Simulations = ['Walk']
pp=[]
rr=[]
for simulation in Simulations:
    all_fms = preprocess_fms_data(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\{}\Anonymous_FMS_All.csv".format(simulation))
    files = glob.glob(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\{}\*\*\eye*".format(simulation))

    for path in files:
        try:
            eye_data = preprocess_eye_tracking_data(path)
            updated_eye,fms_p1 = match_eye_tracking_with_fms(eye_data, all_fms)
            print(updated_eye.shape)
            if 'FMS' in updated_eye.columns:
              print(np.unique(updated_eye.FMS))
            else : 
                pp.append(updated_eye)
                rr.append(fms_p1)


            # print('shape of Eye data with fms',updated_eye.shape)
           

        except Exception as e:
            print(f"An error occurred while processing the eye-tracking file {path}: {e}")


(10974, 45)
[1. 2.]
(10948, 45)
[0. 1.]
(10514, 45)
[1. 2. 3.]
(11415, 45)
[0. 1.]
(11011, 45)
[0. 1. 2.]
(11293, 45)
[2. 3.]
(10724, 45)
[0. 1.]
(11383, 45)
[1. 2. 3.]
(11379, 45)
[0.]
(11582, 45)
[0. 1.]
(11494, 45)
[0. 1.]
(10570, 45)
[0. 2.]
(11382, 45)
[1. 2.]
(11063, 45)
[0.]
(11408, 45)
[0.]
(11307, 45)
[0.]
(11165, 45)
[0.]
(8243, 45)
[0.]
(11382, 45)
[0.]
(11392, 45)
[0.]
(11034, 45)
[0. 1. 2.]
(11336, 45)
[0. 1. 2. 3.]
(11078, 45)
[1. 2. 3.]
(11518, 45)
[1.]


In [95]:
import os
import pandas as pd
import glob


Simulations = ['Walk', 'Room', 'Roller', 'Sea', 'Beach']

for simulation in Simulations:
    all_fms = preprocess_fms_data(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\{}\Anonymous_FMS_All.csv".format(simulation))    
    files = glob.glob(r"C:\Users\purna\OneDrive\Desktop\Cybersickness_Shovon\{}\*\*\eye*".format(simulation))
    
    output_dir = os.path.join(r"D:\Save Lab\Shovons_data", simulation, "Processed_Eye_Data")
    os.makedirs(output_dir, exist_ok=True)
    participant_file_count = {}

    for path in files:
        try:
            eye_data = preprocess_eye_tracking_data(path)
            updated_eye,_ = match_eye_tracking_with_fms(eye_data, all_fms)
            
            # Extract the participant number from the 'participant' column
            participant_num = updated_eye['participant'].iloc[0]
            
         
            if participant_num not in participant_file_count:
                participant_file_count[participant_num] = 1
            else:
                participant_file_count[participant_num] += 1
            
            # Check if the 'FMS' column exists before saving
            if 'FMS' in updated_eye.columns:
                output_file = os.path.join(output_dir, f"sub{participant_num}_eye_{participant_file_count[participant_num]}.csv")
                updated_eye.to_csv(output_file, index=False)
                print(f"Saved updated eye data for participant {participant_num} to {output_file}")
            else:
                print(f"Skipping file {path} as 'FMS' column is not present in the DataFrame.")

        except Exception as e:
            print(f"An error occurred while processing the eye-tracking file {path}: {e}")


Saved updated eye data for participant 1.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub1.0_eye_1.csv
Saved updated eye data for participant 2.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub2.0_eye_1.csv
Saved updated eye data for participant 3.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub3.0_eye_1.csv
Saved updated eye data for participant 4.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub4.0_eye_1.csv
Saved updated eye data for participant 5.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub5.0_eye_1.csv
Saved updated eye data for participant 6.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub6.0_eye_1.csv
Saved updated eye data for participant 7.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub7.0_eye_1.csv
Saved updated eye data for participant 8.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub8.0_eye_1.csv
Saved updated eye data for participant 9.0 to D:\Save Lab\Shovons_data\Walk\Processed_Eye_Data\sub9.0_ey